In [ ]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Dead-simple calculator: No validation BS, just eval (safe for demo; don't use in prod)
def calc(expr: str) -> str:
    try:
        # Super basic: Assume clean input like '25*4-10'
        return str(eval(expr.replace(' ', '')))  # Strip spaces for safety
    except:
        return "Calc failed—try simpler math."

calc_tool = Tool(
    name="Calc",
    description="Do math like '25*4-10'.",
    func=calc,
)

# LLM (try temperature=0 for consistency)
llm = Ollama(model="mistral", temperature=0)  # Or swap to "llama3.2:1b" for speed

# Built-in agent—no custom prompt needed
agent = initialize_agent(
    tools=[calc_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,  # See the steps, but it'll be short
    max_iterations=2,  # Hard cap to prevent loops
    early_stopping_method="generate",
    handle_parsing_errors=True,  # Graceful fails
)
# Run (should finish quick)
result = agent.run("What is 25 times 4 minus 10? Use the tool.")
print("Answer:", result)

In [ ]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts import PromptTemplate

# Search tool
search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="WebSearch",
    description="Useful for getting current info from the web. Input: your search query.",
    func=search.run,
)

# Initialize LLM
llm = Ollama(model="mistral", temperature=0)

# ReAct prompt (same as Demo 1)
react_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
    template="""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""
)

# Create agent
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
)

# Run
result = agent.run("What's the current CEO of xAI as of October 2025?")
print(result)

In [2]:
import yfinance as yf
from ollama import Client  # For Ollama integration

# Step 1: Fetch stock data (e.g., Apple - AAPL)
ticker = "AAPL"  # Change to any symbol like "GOOGL" or "TSLA"
stock = yf.Ticker(ticker)
info = stock.info
current_price = info.get('currentPrice', 'N/A')
market_cap = info.get('marketCap', 'N/A')
change = info.get('regularMarketChangePercent', 'N/A')

# Format data
data = f"Stock: {ticker}\nCurrent Price: ${current_price}\nMarket Cap: ${market_cap:,}\nDaily Change: {change}%"

print("Raw Data:\n", data)

# Step 2: Query Ollama Mistral for a quick analysis
ollama = Client(host='http://localhost:11434')  # Your local Ollama
prompt = f"Based on this stock data: {data}. Give a 1-sentence summary: Is it a good time to buy or sell? Why?"
response = ollama.generate(model='mistral', prompt=prompt)

print("\nOllama's Take:\n", response['response'])

Raw Data:
 Stock: AAPL
Current Price: $271.4
Market Cap: $4,027,681,603,584
Daily Change: 0.630323%

Ollama's Take:
  Given the current daily change of 0.630323% for AAPL stock and considering market capitalization, this slight increase might indicate a potential growth trend. However, whether it's a good time to buy or sell depends on individual investment strategies, financial goals, and risk tolerance. It is always recommended to conduct further research or consult with a financial advisor before making any investment decisions.


In [ ]:
#!pip install yfinance

In [ ]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.prompts import PromptTemplate

# Sample CSV creation (run once; delete after)
data = {'product': ['A', 'B', 'C'], 'sales': [100, 200, 150], 'region': ['North', 'South', 'North']}
pd.DataFrame(data).to_csv('sales.csv', index=False)

# CSV reader tool
def read_csv(query: str) -> str:
    try:
        df = pd.read_csv('sales.csv')
        # Simple query handling (e.g., total sales, avg by region)
        if 'total sales' in query.lower():
            return f"Total sales: {df['sales'].sum()}"
        elif 'average sales' in query.lower():
            return f"Average sales: {df['sales'].mean():.2f}"
        elif 'north region' in query.lower():
            north_sales = df[df['region'] == 'North']['sales'].sum()
            return f"North region sales: {north_sales}"
        else:
            return f"Data summary: {df.to_string()}"
    except Exception as e:
        return f"Error reading CSV: {str(e)}"

csv_tool = Tool(
    name="CSVReader",
    description="Useful for analyzing sales.csv data. Input: query like 'total sales' or 'average sales' or 'north region sales'.",
    func=read_csv,
)

# Initialize LLM
llm = Ollama(model="mistral", temperature=0)

# ReAct prompt (same as previous)
react_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
    template="""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""
)

# Create agent
agent = initialize_agent(
    tools=[csv_tool],
    llm=llm,
    agent=AgentType.REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
)

# Run
result = agent.run("What's the total sales from the north region?")
print(result)